### Chapter 18
**CH18 Forecasting daily ticket sales for a swimming pool**

using swim data

version 1.1 2024-01-10

In [ ]:
import pandas as pd
import numpy as np
import warnings
import sys
import os

# import pandas_market_calendars as 
import holidays
from datetime import datetime
from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns
from mizani.formatters import date_format
from patsy import dmatrices
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline

### Get Data

In [ ]:
# Current script and repository folder
current_path = os.getcwd()
repository_path = current_path.split('Ch18')[0]

In [ ]:
# Add utils folder to sys path 
# Note: os.path.join() creates a string with the right syntax for defining a path for your operating sytem.
sys.path.append(os.path.join(repository_path, 'utils'))

In [ ]:
# Define data folder
data_path = os.path.join(repository_path, 'data')

In [ ]:
# Import the prewritten helper functions
from py_helper_functions import *

In [ ]:
# DATA IMPORT - FROM GITHUB
data = pd.read_csv('https://raw.githubusercontent.com/peterduronelly/DA3-Python-Codes/main/data/swim_work.csv')

In [ ]:
data.head()

In [ ]:
data.info()

### EDA

In [ ]:
daily_agg = data.copy()

In [ ]:
daily_agg.date = pd.to_datetime(daily_agg.date, format= '%Y-%m-%d')

In [ ]:
daily_agg.info()

In [ ]:
daily_agg["year"] = daily_agg["date"].dt.year
daily_agg["quarter"] = daily_agg["date"].dt.quarter
daily_agg["month"] = daily_agg["date"].dt.month
daily_agg["day"] = daily_agg["date"].dt.day
daily_agg["dow"] = daily_agg["date"].dt.dayofweek + 1
daily_agg["weekend"] = daily_agg["dow"].isin([6, 7])

In [ ]:
daily_agg["school_off"] = (
    ((daily_agg["day"] > 15) & (daily_agg["month"] == 5) & (daily_agg["day"] <= 30))
    | ((daily_agg["month"] == 6) | (daily_agg["month"] == 7))
    | ((daily_agg["day"] < 15) & (daily_agg["month"] == 8))
    | ((daily_agg["day"] > 20) & (daily_agg["month"] == 12))
)

In [ ]:
daily_agg["trend"] = daily_agg.index + 1

In [ ]:
# Get holiday calendar ----------------------------------

In [ ]:
minyear = daily_agg.date.min().year
maxyear = daily_agg.date.max().year

In [ ]:
usholidays = holidays.UnitedStates(years = [x for x in range(minyear, maxyear + 1, 1)])

In [ ]:
for dat in usholidays.items():
    print(dat)

In [ ]:
holiday_days = [x[0] for x in usholidays.items()]

In [ ]:
holiday_days[0:10]

In [ ]:
daily_agg.date[0]

In [ ]:
daily_agg["isHoliday"] = daily_agg["date"].isin(holiday_days)

Did we get the holidays right?

In [ ]:
daily_agg[daily_agg.date == datetime(2010,7,4)]

Is *Maria Himmelfahrt* a holiday in the US?

In [ ]:
daily_agg[daily_agg.date == datetime(2010,8,15)]

### Define vars for analysis

In [ ]:
daily_agg["q_month"] = daily_agg.groupby("month")["QUANTITY"].transform("mean")

daily_agg["QUANTITY2"] = np.where(daily_agg["QUANTITY"] < 1, 1, daily_agg["QUANTITY"])

daily_agg["q_ln"] = np.log(daily_agg["QUANTITY2"])

daily_agg["tickets"] = daily_agg.groupby(["month", "dow"])["QUANTITY"].transform("mean")

daily_agg["tickets_ln"] = daily_agg.groupby(["month", "dow"])["q_ln"].transform("mean")

daily_agg["dow_abb"] = daily_agg["date"].dt.day_name().str[:3]

daily_agg["month_abb"] = daily_agg["date"].dt.month_name().str[:3]

## Descriptive graphs

In [ ]:
ggplot(
    daily_agg.loc[daily_agg.year == 2015, :], aes(x="date", y="QUANTITY")
) + geom_line(size=0.4, color=color[0]) + scale_x_date(
    breaks=["2015-01-01", "2015-04-01", "2015-07-01", "2015-10-01", "2016-01-01"],
    labels=date_format("%d%b%Y"),
    date_minor_breaks="1 month",
) + labs(
    x="Date (day)", y="Daily ticket sales"
) + theme_bw()

In [ ]:
daily_agg[daily_agg.year == 2015].plot(
    kind = 'line', figsize = (8,6),
    x = 'date', y = 'QUANTITY', 
    grid = True, legend = False, title = 'Daily ticket sales in 2015');

In [ ]:
ggplot(
    daily_agg.loc[(daily_agg.year >= 2010) & (daily_agg.year <= 2014), :],
    aes(x="date", y="QUANTITY"),
) + geom_line(size=0.2, color=color[0]) + scale_x_date(
    breaks=[
        "2010-01-01",
        "2011-01-01",
        "2012-01-01",
        "2013-01-01",
        "2014-01-01",
        "2015-01-01",
    ],
    labels=date_format("%d%b%Y"),
    date_minor_breaks="3 months",
) + labs(
    x="Date (day)", y="Daily ticket sales"
) + theme_bw()

In [ ]:
daily_agg[daily_agg.year < 2016].plot(
    kind = 'line', figsize = (8,6),
    x = 'date', y = 'QUANTITY', linewidth = 0.5,
    grid = True, legend = False, title = 'Daily ticket sales between 2010-2015');

In [ ]:
ggplot(daily_agg, aes(x="reorder(month_abb,month)", y="QUANTITY")) + geom_boxplot(
    color=color[0],size=0.8, outlier_stroke=0.4, outlier_color="yellow", outlier_alpha=0.6
) + labs(x="Date (month)", y="Daily ticket sales") + theme_bw()

In [ ]:
ax = sns.boxplot(data= daily_agg, x = 'month', y = 'QUANTITY')
ax.set_ylabel('Daily ticket sales')
ax.set_title('Ticket sales distribution by month');

In [ ]:
ggplot(
    daily_agg, aes(x="reorder(dow_abb,dow)", y="QUANTITY")
) + geom_boxplot(
    color=color[0],size=0.8, outlier_stroke=0.4, outlier_color="yellow", outlier_alpha=0.6
) + labs(x="Day of the week", y="Daily ticket sales", title = 'Ticket sales distribution by day of week'
) + theme_bw()

In [ ]:
ax = sns.boxplot(data= daily_agg, x = 'dow', y = 'QUANTITY', color = 'grey')
ax.set_ylabel('Daily ticket sales')
ax.set_title('Ticket sales distribution by day of week');

In [ ]:
# to check for interactions, look at the heatmap

swim_heatmap = (
    ggplot(
        daily_agg,
        aes(x="reorder(dow_abb,dow)", y="reorder(month_abb,month)", fill="tickets"),
    )
    + geom_tile(colour="white")
    + scale_fill_cmap(trans="reverse")
    + labs(x="Day of the week", y="Month") 
    + theme_bw()
    + theme(
        legend_position="right",
        legend_text=element_text(size=10),
        legend_title=element_text(size=10),
    )
)
swim_heatmap

In [ ]:
daily_agg.pivot_table(index="month", columns="dow", values="QUANTITY", aggfunc='sum')

In [ ]:
daily_agg.pivot_table(index="month", columns="dow", values="QUANTITY", aggfunc='mean')

In [ ]:
sns.heatmap(
    data = daily_agg.pivot_table(index="month", columns="dow", values="QUANTITY", aggfunc='mean'), 
    annot = True, 
    # which colormap do you prefer?
    cmap = 'turbo',
    # cmap = 'coolwarm',
    fmt = '.0f');

`matplotlib` colormaps [here](https://matplotlib.org/stable/users/explain/colors/colormaps.html)

In [ ]:
swim_heatmap_log = (
    ggplot(
        daily_agg,
        aes(x="reorder(dow_abb,dow)", y="reorder(month_abb,month)", fill="tickets_ln"),
    )
    + geom_tile(colour="white")
    + scale_fill_cmap(trans="reverse")
    + labs(x="Day of the week", y="Month") 
    + theme_bw()
    + theme(
        legend_position="right",
        legend_text=element_text(size=10),
        legend_title=element_text(size=10),
    )
)
swim_heatmap_log

### Prediction

#### Create train/holdout data

In [ ]:
factor_cols = ["month", "dow", "isHoliday", "school_off"]

daily_agg[factor_cols] = daily_agg[factor_cols].astype("category")
data_holdout = daily_agg.loc[daily_agg['year']==2016,:]
data_train = daily_agg.loc[daily_agg['year']<2016,:]

In [ ]:
data_train.tail()

In [ ]:
logo = LeaveOneGroupOut()
groups = data_train.loc[:,'year'].to_numpy()
groups

Note: `LeaveOneOut()` is equivalent to `KFold(n_splits=n)`

In [ ]:
data_train.year.unique()

#### Linear regression

In [ ]:
lin_reg = LinearRegression(fit_intercept=False)

LeaveOneGroupOut object's `split` method:
- **X**: array-like of shape (n_samples, n_features); training data, where n_samples is the number of samples and n_features is the number of features.
- **y**: object; always ignored, exists for compatibility.
- **groups**: object; always ignored, exists for compatibility.

In [ ]:
def fit_cv_model_get_rmse(y, X, groups):
    rmse_folds = []
    for train_index, test_index in logo.split(X, y, groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        lin_reg.fit(X_train, y_train)
        y_hat = lin_reg.predict(X_test)
        rmse_folds.append(mean_squared_error(y_test, y_hat, squared=False))

    return np.mean(rmse_folds)

**Model 1: linear trend + monthly seasonality**

In [ ]:
%%time
y, X = dmatrices("QUANTITY ~ 1+ trend + month", data_train)

rmse_reg1 = fit_cv_model_get_rmse(y, X, groups)

**Model 2: linear trend + monthly seasonality + days of week seasonality**

In [ ]:
y,X = dmatrices("QUANTITY ~ 1+ trend + month + dow",data_train)

rmse_reg2 = fit_cv_model_get_rmse(y, X, groups)

**Model 3: linear trend + monthly seasonality + days of week  seasonality + holidays**

In [ ]:
y,X = dmatrices("QUANTITY ~ 1 + trend + month + dow + isHoliday",data_train)

rmse_reg3 = fit_cv_model_get_rmse(y, X, groups)

**Model 4: linear trend + monthly seasonality + days of week  seasonality + holidays + sch$*$dow**

In [ ]:
y,X = dmatrices("QUANTITY ~ 1 + trend + month + dow + isHoliday + school_off*dow",data_train)

rmse_reg4 = fit_cv_model_get_rmse(y, X, groups)

**Model 5: linear trend + monthly seasonality + days of week  seasonality + holidays + interactions**

In [ ]:
y, X = dmatrices(
    "QUANTITY ~ 1 + trend + month + dow + isHoliday + school_off*dow+ weekend*month",
    data_train,
)

rmse_reg5 = fit_cv_model_get_rmse(y, X, groups)

In [ ]:
data_train2 = data_train[data_train.QUANTITY >= 1]
groups = data_train2.loc[:,'year'].to_numpy()

Note: we could have done:
```python
groups = data_train2.year.to_numpy()
```

**Model 6: trend + monthly seasonality + days of week seasonality + holidays + interactions**

Why is it different than Model 5?

In [ ]:
y, X = dmatrices(
    "q_ln ~ 1 + trend + month + dow +school_off*dow", data_train2
)

rmse_folds = []
for train_index, test_index in logo.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lin_reg.fit(X_train, y_train)
    y_hat = lin_reg.predict(X)

    corrb = mean_squared_error(y , y_hat)

    y_hat = np.exp((lin_reg.predict(X_test) + corrb / 2))
    rmse_folds.append(mean_squared_error(np.exp(y_test), y_hat, squared=False))

rmse_reg6 = np.mean(rmse_folds)
# rmse_reg6

#### Prophet

Cross-validation with `Prophet` done with prophet: https://facebook.github.io/prophet/docs/diagnostics.html. This is a *time-series-based (!!!)* cross-validation.

In [ ]:
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

*Question*: why are we building an additive model?

In [ ]:
model_prophet = Prophet(
    seasonality_mode="additive",
    yearly_seasonality="auto",
    weekly_seasonality="auto",
    growth="linear",
    daily_seasonality=True,
)

model_prophet = Prophet.add_country_holidays(model_prophet,"US")

In [ ]:
model_prophet = Prophet.fit(
    model_prophet,
    df=data_train[["date", "QUANTITY"]].rename({"date": "ds", "QUANTITY": "y"}, axis=1),
)

In [ ]:
cv_pred = cross_validation(
    model_prophet, 
    initial="365 days", 
    period="365 days", 
    horizon="365 days"
)

In [ ]:
cv_pred

In [ ]:
performance_metrics(cv_pred,rolling_window = 1)

In [ ]:
rmse_prophet_cv = performance_metrics(cv_pred, rolling_window = 1)["rmse"][0]

Note: M6 log model rmse is slightly different from the one found in the book

In [ ]:
pd.DataFrame(
    [rmse_reg1, rmse_reg2, rmse_reg3, rmse_reg4, rmse_reg5, rmse_reg6, rmse_prophet_cv],
    ["M" + str(i) for i in range(1, 6)] + ["M6 (log)", "M7 (Prophet)"],
    columns=["RMSE"],
).round(2)

### Evaluate best model on the holdout set

In [ ]:
lin_reg = LinearRegression(fit_intercept=False)

y, X = dmatrices(
    "QUANTITY ~ 1 + trend + month + dow + isHoliday + school_off*dow+ weekend*month",
    data_train,
)

lin_reg.fit(X, y)

_, X_holdout = dmatrices(
    "QUANTITY ~ 1 + trend + month + dow + isHoliday + school_off*dow+ weekend*month",
    data_holdout,
)

*Question*: what is '`_`' in the previous code chunk?

In [ ]:
data_holdout["y_hat_5"] = lin_reg.predict(X_holdout)

In [ ]:
rmse_holdout_best = mean_squared_error(
    data_holdout.QUANTITY, 
    data_holdout.y_hat_5, 
    squared=False # default: True > returns MSE
)
rmse_holdout_best

#### Detour: interpreting regression coefficients

In [ ]:
y, X = dmatrices(
    "QUANTITY ~ trend + month + dow + isHoliday + school_off*dow+ weekend*month",
    data_train,
)

In [ ]:
best_ols_results = sm.OLS(y, X).fit()

In [ ]:
print(best_ols_results.summary2())

<br>

- What does it mean?

```
"The smallest eigenvalue is 6.54e-28. This might indicate that there are 
strong multicollinearity problems or that the design matrix is singular" 

```
.

In [ ]:
best_ols_results.eigenvals

In [ ]:
plt.hist(best_ols_results.resid, bins = 51, rwidth = 0.9);

### Plot best predictions

Relative RMSE on the holdout set per month

In [ ]:
group = data_holdout.sort_values(by=["month"]).groupby("month")

In [ ]:
type(group)

In [ ]:
group.apply(lambda x: mean_squared_error(x.QUANTITY, x.y_hat_5, squared=False))

In [ ]:
rmse_monthly = pd.DataFrame(
    [
        group.apply(lambda x: mean_squared_error(x.QUANTITY, x.y_hat_5, squared=False)),
        group.apply(
            lambda x: mean_squared_error(x.QUANTITY, x.y_hat_5, squared=False) / np.mean(x.QUANTITY)
        ),
    ],
    index=["RMSE", "RMSE_norm"],
).T.reset_index()

In [ ]:
rmse_monthly

In [ ]:
g_predictions_rmse = (
    ggplot(rmse_monthly, aes(x="month", y="RMSE_norm"))
    + geom_col(color=color[0],fill=color[0])
    + labs(x="Date (month)", y="RMSE (normalized by monthly sales)")
    + theme_bw()
)
g_predictions_rmse

In [ ]:
rmse_monthly.plot(
    kind = 'bar', x = 'month', y = 'RMSE_norm',
    legend = False
);

In [ ]:
ax = sns.barplot(data = rmse_monthly, x = 'month', y = 'RMSE_norm')
ax.grid(visible = True, axis = 'y')
ax.set_ylabel('normalized RMSE')
ax.set_title('Normalized RMSE across months');

In [ ]:
(
    ggplot(data_holdout, aes(x="date"))
    + geom_line(aes(y="QUANTITY"), color=color[0], linetype="solid")
    + geom_line(aes(y="y_hat_5"), color=color[1], linetype="dashed")
    + scale_y_continuous(expand=(0, 0))
    + scale_x_date(
        expand=(0, 0),
        breaks=[
            "2016-01-01",
            "2016-03-01",
            "2016-05-01",
            "2016-07-01",
            "2016-09-01",
            "2016-11-01",
            "2017-01-01",
        ],
        labels=date_format("%d%b%Y"),
        date_minor_breaks="1 month",
    )
    + scale_linetype_manual(name="", values=("solid", "twodash"))
    + labs(x="Date (day)", y="Daily ticket sales")
    + scale_fill_identity(name="", breaks=color[0:2], labels=["Actual", "Predicted"])
    + theme_bw()
)

In [ ]:
(
    ggplot(data_holdout.query("month == 8"), aes(x="date"))
    + geom_line(aes(y="QUANTITY"), color=color[0], size=1)
    + geom_line(aes(y="y_hat_5"), color=color[1], linetype="dashed", size=1)
    + geom_ribbon(aes(ymin="QUANTITY", ymax="y_hat_5"), fill="yellow", alpha=0.3)
    + labs(y = 'tickets sold', title = 'Actual vs predicted ticket sales, August 2016')
    + scale_y_continuous(expand=(0.01, 0.01), limits=(0, 150))
    + scale_x_date(
        expand=(0.01, 0.01),
        breaks=["2016-08-01", "2016-08-08", "2016-08-15", "2016-08-22", "2016-08-29"],
        labels=date_format("%d%b"),
    )
    + theme_bw()
)

In [ ]:
x = data_holdout[(data_holdout.year == 2016) & (data_holdout.month == 8)].date
y1 = data_holdout[(data_holdout.year == 2016) & (data_holdout.month == 8)].QUANTITY
y2 = data_holdout[(data_holdout.year == 2016) & (data_holdout.month == 8)].y_hat_5

In [ ]:
plt.subplots(figsize = (8,6))
plt.plot(x, y1, color = 'k')
plt.plot(x, y2, color = 'k', linestyle = '--')
plt.fill_between(x, y1, y2, color = 'lightblue')
plt.legend(['actual', 'predicted'])
plt.ylabel('tickets sold')
plt.grid(True)
plt.title('Actual vs predicted ticket sales, August 2016')
plt.xticks(x.tolist()[0::7]);